In [1]:
#mount
import os
cwd = os.getcwd()
if cwd[0:2] != "C:":
    from google.colab import drive
    drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [2]:
import sys
root_dir="/content/gdrive/My Drive/Projects/flower"
sys.path.append(root_dir)

%cd "/content/gdrive/My Drive/Projects/flower/wm"

/content/gdrive/My Drive/Projects/flower/wm


In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
import pandas as pd
import numpy as np
import datetime as dt
import importlib
import tradestats
import pandas as pd
from datetime import datetime

In [5]:
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import graphviz

In [6]:
importlib.reload(tradestats)
from tradestats import *

In [8]:
prices = loaddata_1W('mbk_w.csv')

In [19]:
importlib.reload(tradestats)
from tradestats import *
from tradestats import rose
periods = [1]
dictList = []
colFeat = []
keyList = []
roseDict   = rose(prices, periods)
dictList.append(roseDict.df); colFeat.append('rose'); keyList.append(periods)

In [20]:
# prices.dropna(inplace=True)
masterFrame = pd.DataFrame(index=prices.index)
masterFrame[['id','year','month','day','date','weekday','open','high','low','close','volume']] = prices[['id','year','month','day','date','weekday','open','high','low','close','volume']]

In [21]:
for i in range(0,len(dictList)):#iteracja po funkcjach
    for j in keyList[i]: #iteracja po okresach
        for k in list(dictList[i][j]): #iteracja po kolumnach danej funkcji w okresie
            colID = colFeat[i] + str(j) + k
            masterFrame[colID] = dictList[i][j][k]

In [ ]:
# masterFrame.to_csv(sep=';',path_or_buf='../Data/masterFrame.csv',date_format="%Y-%m-%d",index = False,na_rep='')

In [22]:
masterFrame = masterFrame.dropna()

In [23]:
masterFrame#.tail(5)

,id,year,month,day,date,weekday,open,high,low,close,volume,rose1open,rose1high,rose1low,rose1close,rose1open_prev,rose1high_prev,rose1low_prev,rose1close_prev,rose1rose
1,1,1992,10,18,1992-10-18,6,3.540021,3.600065,3.540021,3.600065,202516.798888,3.540021,3.600065,3.540021,3.600065,4.059913,4.059913,3.919933,3.919933,-1
2,2,1992,10,25,1992-10-25,6,3.959962,4.339995,3.959962,4.339995,215293.731485,3.959962,4.339995,3.959962,4.339995,3.540021,3.600065,3.540021,3.600065,1
3,3,1992,11,1,1992-11-01,6,4.399917,4.399917,4.220030,4.220030,182786.422438,4.399917,4.399917,4.220030,4.220030,3.959962,4.339995,3.959962,4.339995,0
4,4,1992,11,8,1992-11-08,6,4.220030,4.220030,4.200015,4.200015,87812.834962,4.220030,4.220030,4.200015,4.200015,4.399917,4.399917,4.220030,4.220030,-1
5,5,1992,11,15,1992-11-15,6,4.240044,4.240044,4.240044,4.240044,86060.953868,4.240044,4.240044,4.240044,4.240044,4.220030,4.220030,4.200015,4.200015,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1630,1630,2024,1,7,2024-01-07,6,533.000000,535.000000,506.000000,525.600000,126561.000000,533.000000,535.000000,506.000000,525.600000,549.000000,556.000000,526.600000,535.000000,-1
1631,1631,2024,1,14,2024-01-14,6,528.400000,543.400000,511.200000,526.600000,119734.000000,528.400000,543.400000,511.200000,526.600000,533.000000,535.000000,506.000000,525.600000,0
1632,1632,2024,1,21,2024-01-21,6,527.000000,530.800000,488.800000,499.300000,102028.000000,527.000000,530.800000,488.800000,499.300000,528.400000,543.400000,511.200000,526.600000,-1
1633,1633,2024,1,28,2024-01-28,6,505.400000,511.400000,488.100000,506.800000,103159.000000,505.400000,511.400000,488.100000,506.800000,527.000000,530.800000,488.800000,499.300000,0


In [ ]:
importlib.reload(tradestats)
from tradestats import *
starttime = datetime.now()
# alltrades = preparetrades_brut_tsl(masterFrame,[1],[0.001,0.002,0.003,0.004,0.005,0.006],[0.001,0.002,0.003,0.004,0.005,0.006],[0.001,0.002,0.003,0.004,0.005,0.006],2016,2017)
alltrades = preparetrades_brut_tp(masterFrame,[1],[0.001,0.002,0.003,0.004,0.005,0.006],[0.001,0.002,0.003,0.004,0.005,0.006],2016,2017)
endtime = datetime.now()
print(str(endtime - starttime))

In [ ]:
importlib.reload(tradestats)
from tradestats import *
trades = cleartrades_brut(alltrades.copy(),True)
# trades,alltrades = preparetrades(masterFrame,[-1,1],[5],[13],[0.002],True)

In [ ]:
#____________________________________________________________________________________________________________________
#
#
alltrades.head(2)

In [ ]:
importlib.reload(tradestats)
from tradestats import *
stats1 = runstats_brut_v3(alltrades)